In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1691454429219_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import boto3
import os
import sys
import pytz
import numpy as np
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from datetime import datetime 
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
agora=datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc=agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession.builder.appName('trusted_orders').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
_DIA_ = '20230804'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
bucket_refined = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/004_refined'.format(dia=_DIA_)
bucket_trusted = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/003_trusted'.format(dia=_DIA_)
bucket_raw = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/002_raw'.format(dia=_DIA_)
bucket_control = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/005_control'.format(dia=_DIA_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
output_refined = 't_indicadores_valor_medio_emprest_e_dist_idade'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
path_lake = os.path.join('s3://',bucket_trusted,'trusted_application_train_20230805')
application_train_00=sqlContext.read.format("parquet").load(path_lake)
application_train_00.registerTempTable("application_train_00")
application_train_00.cache()
application_train_00.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

307511

In [25]:
application_train_00.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ref: integer (nullable = true)
 |-- ts_proc: long (nullable = true)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nullabl

In [26]:
indicadores_00 = spark.sql(     
    """
        SELECT
            SK_ID_CURR,
            TARGET,
            AMT_CREDIT,
            DAYS_BIRTH,
            FLOOR(DAYS_BIRTH / 365.25 * (-1)) AS IDADE
            
        FROM 
           application_train_00
           
    """)
indicadores_00.registerTempTable("indicadores_00")
indicadores_00.cache()
indicadores_00.count() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

307511

In [27]:
indicadores_00.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+----------+----------+-----+
|SK_ID_CURR|TARGET|AMT_CREDIT|DAYS_BIRTH|IDADE|
+----------+------+----------+----------+-----+
|    100002|     1|  406597.5|     -9461|   25|
|    100003|     0| 1293502.5|    -16765|   45|
|    100004|     0|  135000.0|    -19046|   52|
|    100006|     0|  312682.5|    -19005|   52|
|    100007|     0|  513000.0|    -19932|   54|
|    100008|     0|  490495.5|    -16941|   46|
|    100009|     0| 1560726.0|    -13778|   37|
|    100010|     0| 1530000.0|    -18850|   51|
|    100011|     0| 1019610.0|    -20099|   55|
|    100012|     0|  405000.0|    -14469|   39|
|    100014|     0|  652500.0|    -10197|   27|
|    100015|     0|  148365.0|    -20417|   55|
|    100016|     0|   80865.0|    -13439|   36|
|    100017|     0|  918468.0|    -14086|   38|
|    100018|     0|  773680.5|    -14583|   39|
|    100019|     0|  299772.0|     -8728|   23|
|    100020|     0|  509602.5|    -12931|   35|
|    100021|     0|  270000.0|     -9776

In [28]:
tabela = output_refined

path_athena = os.path.join('s3://',bucket_refined,tabela)
print('path_tabela:', path_athena)
indicadores_00.coalesce(1).write.\
format("parquet").mode("overwrite").option("compression", "gzip").option("overwriteSchema", "true").\
option("path", path_athena).saveAsTable(tabela)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

path_tabela: s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/004_refined/t_indicadores_valor_medio_emprest_e_dist_idade

### cria tabela athena

In [22]:
# Criando External Table
import boto3
client = boto3.client('athena',region_name='us-east-1')
config = {
    'OutputLocation': 's3://bkt-datalake-grupo-01-hackathon-podacademy/athena/log/'
}

sql = """ DROP TABLE IF EXISTS `t_indicadores_valor_medio_emprest_e_dist_idade`"""

context = {'Database': 'pod_academy'}
RES = client.start_query_execution(QueryString = sql, 
                             QueryExecutionContext = context,
                             ResultConfiguration = config)
print(sql)
print(RES)

# Query Execution Parameters
sql = """

CREATE EXTERNAL TABLE `t_indicadores_taxa_inadimplencia`(
  'SK_ID_CURR' bigint,
  'TARGET' bigint,
  'AMT_CREDIT' double,
  'DAYS_BIRTH' bigint,
  'IDADE' bigint
  )
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/004_refined/t_indicadores_valor_medio_emprestimo'
TBLPROPERTIES (
  'has_encrypted_data'='true', 
  'transient_lastDdlTime'='1684958889')   
""".format(dia=_DIA_)

context = {'Database': 'pod_academy'}
client.start_query_execution(QueryString = sql, 
                             QueryExecutionContext = context,
                             ResultConfiguration = config)
print(sql)
print(RES)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred (AccessDeniedException) when calling the StartQueryExecution operation: You are not authorized to perform: athena:StartQueryExecution on the resource. After your AWS administrator or you have updated your permissions, please try again.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/botocore/client.py", line 535, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/usr/local/lib/python3.7/site-packages/botocore/client.py", line 980, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.exceptions.ClientError: An error occurred (AccessDeniedException) when calling the StartQueryExecution operation: You are not authorized to perform: athena:StartQueryExecution on the resource. After your AWS administrator or you have updated your permissions, please try again.

